In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torchvision import transforms
import torch.optim as optim
import random 
from tqdm import tqdm
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn.functional as F
from tqdm.auto import tqdm
from datetime import datetime
import wandb
import time
import os
import re
import math
from nltk.tokenize import word_tokenize
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')
print(nltk.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

3.8.1


[nltk_data] Downloading package punkt to /home/vuda/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
NUM_EPOCHS = 30
BATCH_SIZE = 4
FRAC_SAMPLE = 0.01
MAX_LENGTH_ARTICLE = 512
MIN_LENGTH_ARTICLE = 50
MAX_LENGTH_SUMMARY = 128
MIN_LENGTH_SUMMARY = 20
HIDDEN_DIM = 128
LEARNING_RATE = 0.005
NUM_CYCLES = 3
MAX_PLATEAU_COUNT = 5
WEIGHT_DECAY = 1e-4
model_dir = "../Model"
datafilter = "../dataft"
os.makedirs(datafilter, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

In [3]:
# train_data = pd.read_csv("../dataset/train.csv")
# validation_data = pd.read_csv("../dataset/validation.csv")
# test_data = pd.read_csv("../dataset/test.csv")

# # add col
# train_data.rename(columns={"highlights": "summaries", "article":"articles"}, inplace=True)
# validation_data.rename(columns={"highlights": "summaries","article":"articles"}, inplace=True)
# test_data.rename(columns={"highlights": "summaries", "article":"articles"}, inplace=True)

# train_data["article_word_count"] = train_data["articles"].astype(str).apply(lambda x: len(x.split()))
# train_data["summary_word_count"] = train_data["summaries"].astype(str).apply(lambda x: len(x.split()))

# validation_data["article_word_count"] = validation_data["articles"].astype(str).apply(lambda x: len(x.split()))
# validation_data["summary_word_count"] = validation_data["summaries"].astype(str).apply(lambda x: len(x.split()))

# test_data["article_word_count"] = test_data["articles"].astype(str).apply(lambda x: len(x.split()))
# test_data["summary_word_count"] = test_data["summaries"].astype(str).apply(lambda x: len(x.split()))

# # filter range
# train_data = train_data[
#     (train_data["article_word_count"] <= MAX_LENGTH_ARTICLE) & 
#     (train_data["article_word_count"] >= MIN_LENGTH_ARTICLE) &
#     (train_data["summary_word_count"] <= MAX_LENGTH_SUMMARY) &
#     (train_data["summary_word_count"] >= MIN_LENGTH_SUMMARY)
# ]

# validation_data = validation_data[
#     (validation_data["article_word_count"] <= MAX_LENGTH_ARTICLE) & 
#     (validation_data["article_word_count"] >= MIN_LENGTH_ARTICLE) &
#     (validation_data["summary_word_count"] <= MAX_LENGTH_SUMMARY) &
#     (validation_data["summary_word_count"] >= MIN_LENGTH_SUMMARY)
# ]
# test_data = test_data[
#     (test_data["article_word_count"] <= MAX_LENGTH_ARTICLE) & 
#     (test_data["article_word_count"] >= MIN_LENGTH_ARTICLE) &
#     (test_data["summary_word_count"] <= MAX_LENGTH_SUMMARY) &
#     (test_data["summary_word_count"] >= MIN_LENGTH_SUMMARY)
# ]

# train_sample = train_data.sample(frac=FRAC_SAMPLE, random_state=1)
# validation_sample = validation_data.sample(frac=FRAC_SAMPLE, random_state=1)
# test_sample = test_data.sample(frac=FRAC_SAMPLE, random_state=1)
# train_sample.info()
# print("\n")
# validation_sample.info()
# train_sample.to_csv(os.path.join(datafilter,"train_sample.csv"), index=False)
# test_sample.to_csv(os.path.join(datafilter,"test_sample.csv"), index=False)
# validation_sample.to_csv(os.path.join(datafilter,"validation_sample.csv"), index=False)

In [4]:
# print(tokenize("A dog. in a 'tree with 5.3% rate drop"))

In [5]:
train_sample = pd.read_csv("../dataft/train_sample.csv")
validation_sample = pd.read_csv("../dataft/validation_sample.csv")
test_sample = pd.read_csv("../dataft/test_sample.csv")
train_sample.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960 entries, 0 to 959
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  960 non-null    object
 1   articles            960 non-null    object
 2   summaries           960 non-null    object
 3   article_word_count  960 non-null    int64 
 4   summary_word_count  960 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 37.6+ KB


In [6]:
TOP_K = 100000
EMBEDDING_FILE = "../Embedding/glove.6B.50d.txt"

vocab, embeddings = [], []

with open(EMBEDDING_FILE, 'rt', encoding='utf-8') as ef:
    for i, line in enumerate(ef):
        if i >= TOP_K:
            break
        split_line = line.strip().split(' ')
        i_word = split_line[0]
        i_embeddings = [float(val) for val in split_line[1:]]
        i_embeddings.extend([0.0, 0.0, 0.0, 0.0])  # để dành cho token đặc biệt
        vocab.append(i_word)
        embeddings.append(i_embeddings)


embs_npa = np.array(embeddings)

unk_embedding = np.mean(embs_npa, axis=0).tolist()

dim = embs_npa.shape[1]
sos_embedding = [0.0] * dim
sos_embedding[-3] = 1.0
eos_embedding = [0.0] * dim
eos_embedding[-2] = 1.0
pad_embedding = [0.0] * dim
pad_embedding[-4] = 1.0
# unk_embedding = [0.0] * dim
# unk_embedding[-1] = 1.0

# Update vocab and embeddings
vocab = ["<PAD>", "<SOS>", "<EOS>", "<UNK>"] + vocab
embeddings = [pad_embedding, sos_embedding,
              eos_embedding, unk_embedding] + embeddings

vocab_npa = np.array(vocab)
embs_npa = np.array(embeddings)


def tokenize(text):
    return word_tokenize(text.lower())

def preclean_text(text):
    text = re.sub(r"\s'([a-zA-Z])", r" '\1", text)

    return word_tokenize(text.lower())


stoi_dict = {word: idx for idx, word in enumerate(vocab_npa)}
_unk_idx = stoi_dict["<UNK>"]
itos_dict = {idx: word for idx, word in enumerate(vocab_npa)}

def stoi(string, stoi_dict=stoi_dict):
    return stoi_dict.get(string, _unk_idx)

def itos(idx, itos_dict=itos_dict):
    return itos_dict.get(idx)

def revert_to_text(lst):
    if hasattr(lst, 'tolist'):  # works for both torch.Tensor and np.ndarray
        lst = lst.tolist()
    return [str(itos(int(token))) for token in lst] 


def numericalize(text):
    tokenized_text = tokenize(text)
    return [
        stoi(token)
        for token in tokenized_text
    ]

print(embs_npa.shape[0])
embedding_layer = torch.nn.Embedding.from_pretrained(torch.FloatTensor(embeddings),
                                                     freeze=False,
                                                     padding_idx=stoi("<PAD>"))
embedding_layer.to(device)



100004


Embedding(100004, 54, padding_idx=0)

In [7]:
vocab_size = len(vocab_npa)
print("Embedding shape:", np.array(embeddings).shape) 
print("<PAD> embedding last 4 dims:", embeddings[stoi("<PAD>")][-4:])
print("<SOS> embedding last 4 dims:", embeddings[stoi("<SOS>")][-4:])
print("Word 'the' embedding last 4 dims:", embeddings[stoi("5.3%")])
print(revert_to_text(torch.tensor([0, 1, 2, 3])))

Embedding shape: (100004, 54)
<PAD> embedding last 4 dims: [1.0, 0.0, 0.0, 0.0]
<SOS> embedding last 4 dims: [0.0, 1.0, 0.0, 0.0]
Word 'the' embedding last 4 dims: [0.03656563577138949, -0.18582784663417096, -0.06781056216307739, -0.040585373685092956, -0.07301668493291918, 0.016044644258671987, 0.1372867005388796, -0.07036971064271462, -0.10438737653424474, 0.11584013095302573, 0.03797717741161136, 0.02782290565024015, 0.042134458545775105, 0.006004885678202956, -0.04811175801439264, 0.01938162005008408, -0.06738358746789967, 0.07091858680646929, 0.0485902582917709, 0.12120493521288776, -0.11500593603289162, 0.003967934351382163, 0.05070292498116329, 0.005906024159269963, 0.06472339142162035, 0.022294008330708867, 0.009735554614555037, 0.08725065600012115, -0.042772991751960865, -0.07584176364629841, 0.03525691425508773, 0.04915543195526088, 0.05126815078068994, 0.1762973806305546, 0.02469765032816606, 0.01257729470118004, -0.015610369137945711, 0.016562756680375525, 0.036024580951118

In [8]:
from collections import defaultdict

def analyze_vocab_coverage(sample_data, stoi_dict):
    # Đếm tần suất từ duy nhất
    word_freq = defaultdict(int)

    for text in sample_data['articles'] + sample_data['summaries']:
        tokens = tokenize(text)
        for token in tokens:
            word_freq[token] += 1

    # Phân loại từ vào known / unknown
    known_words = set()
    unknown_words = set()

    for word in word_freq:
        if word in stoi_dict:
            known_words.add(word)
        else:
            unknown_words.add(word)

    total_unique_words = len(known_words) + len(unknown_words)
    coverage = len(known_words) / total_unique_words * 100 if total_unique_words > 0 else 0.0
    print("A word not in dict: ", random.choice(list(unknown_words)))
    return {
        'total_unique_words': total_unique_words,
        'known_unique_words': len(known_words),
        'unknown_unique_words': len(unknown_words),
        'coverage_percentage': coverage,
    }
def print_vocab_stats(name, stats):
    print(f"\n{name} Vocabulary Coverage:")
    print(f"- Unique words: {stats['total_unique_words']}")
    print(f"- Exist in dict: {stats['known_unique_words']}")
    print(f"- Outside the dict: {stats['unknown_unique_words']}")
    print(f"- Coverage rate: {stats['coverage_percentage']:.2f}%")

print_vocab_stats("Train", analyze_vocab_coverage(train_sample, stoi_dict))
print_vocab_stats("Validation", analyze_vocab_coverage(validation_sample, stoi_dict))
print_vocab_stats("Test", analyze_vocab_coverage(test_sample, stoi_dict))


A word not in dict:  finlason

Train Vocabulary Coverage:
- Unique words: 28137
- Exist in dict: 21586
- Outside the dict: 6551
- Coverage rate: 76.72%
A word not in dict:  police-related

Validation Vocabulary Coverage:
- Unique words: 4104
- Exist in dict: 3718
- Outside the dict: 386
- Coverage rate: 90.59%
A word not in dict:  i'am

Test Vocabulary Coverage:
- Unique words: 3707
- Exist in dict: 3368
- Outside the dict: 339
- Coverage rate: 90.86%


In [9]:
class Seq2SeqDataset(Dataset):
    def __init__(self, articles, summaries, stoi, max_len_article=MAX_LENGTH_ARTICLE, max_len_summary=MAX_LENGTH_SUMMARY):
        self.articles = articles  # List of articles
        self.summaries = summaries  # List of summaries
        self.stoi = stoi 
        self.pad_idx = stoi("<PAD>")
        self.sos_idx = stoi("<SOS>")
        self.eos_idx = stoi("<EOS>")
        
        self.max_len_article = max_len_article 
        self.max_len_summary = max_len_summary

    def __len__(self):
        return len(self.articles)

    def __getitem__(self, idx):
        def process_text(text, max_len):
            tokens = [self.sos_idx] + [self.stoi(w) for w in text.split()] + [self.eos_idx]  # Tokenize and add SOS/EOS
            tokens = tokens[:max_len] + [self.pad_idx] * (max_len - len(tokens))  # Pad to max length
            return torch.tensor(tokens), len(tokens)

        article_tokens, article_len = process_text(self.articles[idx], self.max_len_article)
        summary_tokens, summary_len = process_text(self.summaries[idx], self.max_len_summary)
        
        return {
            'article': article_tokens,  # Encoded article
            'article_len': torch.tensor(article_len),
            'summary': summary_tokens,  # Encoded summary
            'summary_len': torch.tensor(summary_len)
        }

def collate_fn(batch):
    # Batch is list os the dict {'article': ..., 'summary': ...}
    return {
        'article': torch.stack([item['article'] for item in batch]),
        'article_len': torch.tensor([item['article_len'] for item in batch]),
        'summary': torch.stack([item['summary'] for item in batch]),
        'summary_len': torch.tensor([item['summary_len'] for item in batch])
    }

# DataLoader setup

torch.set_printoptions(profile="default")
train_dataset = Seq2SeqDataset(train_sample['articles'].tolist(), train_sample['summaries'].tolist(), stoi)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

validation_dataset= Seq2SeqDataset(validation_sample['articles'].tolist(), validation_sample['summaries'].tolist(), stoi)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

test_dataset= Seq2SeqDataset(test_sample['articles'].tolist(), test_sample['summaries'].tolist(), stoi)
test_loader = DataLoader(test_sample, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

# torch.set_printoptions(profile="full")
# print(train_dataset[268]["article"])

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

class SimpleEncoder(nn.Module):
    def __init__(self, embedding_layer, hidden_dim):
        super().__init__()
        self.embedding = embedding_layer
        self.lstm = nn.LSTM(
            input_size=self.embedding.embedding_dim,
            hidden_size=hidden_dim,
            batch_first=True,
            bidirectional=False  # Unidirectional
        )
        self.hidden_dim = hidden_dim
    
    def forward(self, x, seq_lens):
        x = self.embedding(x) 
        
        packed = nn.utils.rnn.pack_padded_sequence(
            input=x,
            lengths=seq_lens.cpu(), 
            batch_first=True,
            enforce_sorted=False 
        )
        packed_output, (hidden, cell) = self.lstm(packed)
        output, _ = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)
        
        # hidden shape: [1, batch_size, hidden_dim]
        return output, hidden.squeeze(0)  # Return only hidden state for decoder init

class SimpleAttention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.attn = nn.Linear(hidden_dim * 2, hidden_dim)
        self.v = nn.Linear(hidden_dim, 1, bias=False)
    
    def forward(self, hidden, encoder_outputs, mask):
        # hidden: [batch_size, hidden_dim]
        # encoder_outputs: [batch_size, seq_len, hidden_dim]
        # mask: [batch_size, seq_len]
        
        hidden = hidden.unsqueeze(1).repeat(1, encoder_outputs.size(1), 1)  # [batch_size, seq_len, hidden_dim]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))  # [batch_size, seq_len, hidden_dim]
        attention = self.v(energy).squeeze(2)  # [batch_size, seq_len]
        
        attention = attention.masked_fill(mask == 0, -1e10)  # Apply mask to padded positions
        return F.softmax(attention, dim=1)  # Normalized attention weights

class SimpleDecoder(nn.Module):
    def __init__(self, embedding_layer, hidden_dim, vocab_size):
        super().__init__()
        self.embedding = embedding_layer
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.attention = SimpleAttention(hidden_dim)
        self.lstm = nn.LSTMCell(self.embedding.embedding_dim + hidden_dim, hidden_dim)
        self.fc_out = nn.Linear(hidden_dim * 2, vocab_size)  # hidden + context
        
    def forward(self, x, hidden, cell, encoder_outputs, mask):
        # x: [batch_size] (long tensor)
        # hidden, cell: [batch_size, hidden_dim]
        # encoder_outputs: [batch_size, seq_len, hidden_dim]
        # mask: [batch_size, seq_len]
        
        x = x.unsqueeze(0) if x.dim() == 0 else x  # handle single example
        embedded = self.embedding(x)  # [batch_size, emb_dim]
        
        # Attention
        attn_weights = self.attention(hidden, encoder_outputs, mask)  # [batch_size, seq_len]
        context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs).squeeze(1)  # [batch_size, hidden_dim]
        
        # LSTM step
        lstm_input = torch.cat((embedded, context), dim=1)  # [batch_size, emb_dim + hidden_dim]
        hidden, cell = self.lstm(lstm_input, (hidden, cell))
        
        # Output prediction
        output = torch.cat((hidden, context), dim=1)  # [batch_size, hidden_dim * 2]
        output = self.fc_out(output)  # [batch_size, vocab_size]
        
        return output, hidden, cell, attn_weights

class Seq2SeqModel(nn.Module):
    def __init__(self, embedding_layer, hidden_dim, vocab_size, pad_idx):
        super().__init__()
        self.encoder = SimpleEncoder(embedding_layer, hidden_dim)
        self.decoder = SimpleDecoder(embedding_layer, hidden_dim, vocab_size)
        self.vocab_size = vocab_size
        self.pad_idx = pad_idx
        self.start_idx = stoi("<SOS>")
    
    def create_mask(self, src):
        # src: [batch_size, seq_len]
        return (src != self.pad_idx)
    
    def forward(self, src, src_lens, trg=None, teacher_forcing_ratio=0.5, max_len=50):
        # src: [batch_size, src_len]
        # src_lens: [batch_size]
        # trg: [batch_size, trg_len] if training, None if inference
        
        batch_size = src.size(0)
        max_len = trg.size(1) if trg is not None else max_len
        
        # Encoder forward
        encoder_outputs, hidden = self.encoder(src, src_lens)  # encoder_outputs: [batch_size, src_len, hidden_dim]
        cell = torch.zeros_like(hidden)  # Initialize cell state for decoder
        
        # Initialize outputs tensor
        outputs = torch.zeros(batch_size, max_len, self.vocab_size).to(src.device)
        
        # First input is <SOS> token
        input = torch.full((batch_size,), self.start_idx, dtype=torch.long, device=src.device)
        
        # Create mask for attention
        mask = self.create_mask(src)
        
        for t in range(max_len):
            output, hidden, cell, _ = self.decoder(
                input, hidden, cell, encoder_outputs, mask
            )
            
            outputs[:, t] = output
            
            # Decide whether to use teacher forcing
            if trg is not None and random.random() < teacher_forcing_ratio:
                input = trg[:, t]
            else:
                input = output.argmax(1)
                
        return outputs

In [11]:
print(f"GPU Memory allocated: {torch.cuda.memory_allocated()/1024**2:.2f} MB")
print(f"GPU Memory reserved: {torch.cuda.memory_reserved()/1024**2:.2f} MB")

GPU Memory allocated: 20.60 MB
GPU Memory reserved: 22.00 MB


In [12]:

model = Seq2SeqModel(
    embedding_layer=embedding_layer,
    hidden_dim=HIDDEN_DIM,
    vocab_size=vocab_size,
    pad_idx=stoi("<PAD>")  # Make sure this matches your vocabulary
).to(device)
print("Embedding shape:", torch.FloatTensor(embeddings).shape)
print("Vocab size:", vocab_size)
print("Model architecture:")
print(model)


Embedding shape: torch.Size([100004, 54])
Vocab size: 100004
Model architecture:
Seq2SeqModel(
  (encoder): SimpleEncoder(
    (embedding): Embedding(100004, 54, padding_idx=0)
    (lstm): LSTM(54, 128, batch_first=True)
  )
  (decoder): SimpleDecoder(
    (embedding): Embedding(100004, 54, padding_idx=0)
    (attention): SimpleAttention(
      (attn): Linear(in_features=256, out_features=128, bias=True)
      (v): Linear(in_features=128, out_features=1, bias=False)
    )
    (lstm): LSTMCell(182, 128)
    (fc_out): Linear(in_features=256, out_features=100004, bias=True)
  )
)


In [13]:
wandb.init(
    project="Seq2Seq-Summarization",
    name=f"seq2seq-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
    config={
        "model": "Seq2Seq-LSTM",
        "hidden_dim": HIDDEN_DIM,
        "batch_size": BATCH_SIZE,
        "learning_rate": LEARNING_RATE,
        "teacher_forcing_ratio": 1.0,
        "vocab_size": len(vocab)
    }
)


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: vubkk67 (vubkk67-hanoi-university-of-science-and-technology). Use `wandb login --relogin` to force relogin


In [14]:
def linear_warmup_decay(step, warmup_steps, total_steps):
    if step < warmup_steps:
        return (step + 1) / (warmup_steps + 1)
    else:
        return max(1e-7, (total_steps - step) / (total_steps - warmup_steps))


def warmup_cosine_with_restarts(step, warmup_steps, total_steps, num_cycles=1):
    if step < warmup_steps:
        return (step + 1) / (warmup_steps + 1)
    else:
        progress = (step - warmup_steps) / (total_steps - warmup_steps)
        cycle_progress = progress * num_cycles % 1
        return max(1e-7, 0.5 * (1 + math.cos(math.pi * cycle_progress)))



def get_scheduler(
    optimizer, total_steps, warmup_steps, num_cycles=None, types='warmup_cosine_with_restarts'
):
    if types == 'warmup_cosine_with_restarts':
        assert num_cycles != None, 'must specify num_cycles when types="warmup_cosine_with_restarts"'
        return torch.optim.lr_scheduler.LambdaLR(
            optimizer,
            lr_lambda=lambda step: warmup_cosine_with_restarts(
                step, warmup_steps, total_steps, num_cycles=num_cycles)
        )
    elif types == 'linear_warmup_decay':
        return torch.optim.lr_scheduler.LambdaLR(
            optimizer,
            lr_lambda=lambda step: linear_warmup_decay(step, warmup_steps, total_steps)
        )
    else:
        raise Exception('not implemented')

In [16]:

def train_model(model, train_loader, optimizer, criterion, device, lr_scheduler):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc="Training", leave=False)
    
    for batch in progress_bar:
        src = batch['article'].to(device)
        src_lens = batch['article_len'].to(device)
        trg = batch['summary'].to(device)
        
        # Forward pass with teacher forcing
        outputs = model(src, src_lens, trg=trg, teacher_forcing_ratio=0.5)  # Using 50% teacher forcing
        
        # Reshape outputs and targets for loss calculation
        # outputs shape: [batch_size, trg_len, vocab_size]
        # trg shape: [batch_size, trg_len]
        outputs = outputs[:, 1:].reshape(-1, outputs.size(-1))  # Skip <SOS> token
        trg = trg[:, 1:].reshape(-1)  # Skip <SOS> token
        
        loss = criterion(outputs, trg)
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        lr_scheduler.step()
        
        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())
    
    return total_loss / len(train_loader)

def evaluate(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0
    progress_bar = tqdm(val_loader, desc="Evaluating", leave=False)
    
    with torch.no_grad():
        for batch in progress_bar:
            src = batch['article'].to(device)
            src_lens = batch['article_len'].to(device)
            trg = batch['summary'].to(device)
            
            # Forward pass without teacher forcing (ratio=0)
            outputs = model(src, src_lens, trg=trg, teacher_forcing_ratio=0)
            
            # Reshape outputs and targets
            outputs = outputs[:, 1:].reshape(-1, outputs.size(-1))
            trg = trg[:, 1:].reshape(-1)
            
            loss = criterion(outputs, trg)
            total_loss += loss.item()
            progress_bar.set_postfix(loss=loss.item())
    
    return total_loss / len(val_loader)


# Save best Model
best_model_path = os.path.join(model_dir, "best_model_300.pth")

# 4. Train loop
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
criterion = nn.CrossEntropyLoss(ignore_index=stoi("<PAD>"))
best_val_loss = float('inf')
# Initialize learning rate scheduler
total_steps = NUM_EPOCHS * len(train_loader)
warmup_steps = int(0.1 * total_steps)  # 10% of total steps for warmup
lr_scheduler = get_scheduler(
    optimizer, 
    total_steps=total_steps,
    warmup_steps=warmup_steps,
    num_cycles=NUM_CYCLES
)
plateau_count = 0
wandb.watch(model)
for epoch in range(NUM_EPOCHS):
    start_time = time.time()
    
    # Train
    train_loss = train_model(model, train_loader, optimizer, criterion, device,lr_scheduler)
    
    # Eval
    val_loss = evaluate(model, validation_loader, criterion, device)
    
    current_lr = lr_scheduler.get_last_lr()[0] if lr_scheduler else LEARNING_RATE
    # Log metrics
    wandb.log({
        "epoch": epoch+1,
        "train_loss": train_loss,
        "val_loss": val_loss,
        "best_val_loss": best_val_loss, 
        "lr": current_lr
    })
    
    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        plateau_count = 0
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'lr_scheduler_state_dict': lr_scheduler.state_dict() if lr_scheduler else None,
        }, best_model_path)
    else:
        plateau_count += 1
    
    print(f"Epoch {epoch+1:02d} | "
          f"Train Loss: {train_loss:.4f} | "
          f"Val Loss: {val_loss:.4f} | "
          f"LR: {current_lr:.6f} | "
          f"Time: {time.time()-start_time:.2f}s")
    
    # Early stopping if validation loss doesn't improve
    if plateau_count >= MAX_PLATEAU_COUNT:
        print(f"Validation loss hasn't improved for {MAX_PLATEAU_COUNT} epochs. Stopping training.")
        break

# Kết thúc W&B
wandb.finish()

Training:   0%|          | 0/240 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/13 [00:00<?, ?it/s]

Epoch 01 | Train Loss: 7.3393 | Val Loss: 5.8663 | LR: 0.001671 | Time: 243.64s


Training:   0%|          | 0/240 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/13 [00:00<?, ?it/s]

...

Epoch 07 | Train Loss: 5.2770 | Val Loss: 5.6535 | LR: 0.002934 | Time: 247.77s


Training:   0%|          | 0/240 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/13 [00:00<?, ?it/s]

Epoch 08 | Train Loss: 5.1919 | Val Loss: 5.6434 | LR: 0.002066 | Time: 245.51s


Training:   0%|          | 0/240 [00:00<?, ?it/s]